In [1]:
%pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: c:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

def search_books(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/book.json' #?query=파이썬&display=100&sort=sim

    # requests get(url, params, headers) 요청 
    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']

    return items_data

def save_json(items_data):
    with open('../data/books.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)

def search_shops(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/shop.json' #?query=파이썬&display=100&sort=sim

    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']
    return items_data

def save_json(items_data):
    with open('../data/books.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)

def save_json_shop(items_data):
    with open('../data/shops.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)


if __name__ == '__main__':
    save_json(search_books('파이썬'))        
    save_json_shop(search_shops('가디건'))

### 리팩토링 된 코드

In [ ]:
import requests
import os
from dotenv import load_dotenv
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}


def search_naver_api(endpoint, query, display=50):
    """네이버 API 검색 함수"""
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }
    url = f'https://openapi.naver.com/v1/search/{endpoint}.json'
    res = requests.get(url, params=payload, headers=headers)
    res.raise_for_status()  # 에러 발생 시 예외 처리
    return res.json().get('items', [])


def save_json(data, filepath):
    """JSON 파일 저장 함수"""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


if __name__ == '__main__':
    books = search_naver_api('book', '파이썬')
    save_json(books, '../data/books.json')

    shops = search_naver_api('shop', '가디건')
    save_json(shops, '../data/shops.json')


In [5]:
import pandas as pd

books_df = pd.read_json('../data/books.json')
print(type(books_df))
books_df.head()

<class 'pandas.core.frame.DataFrame'>


title  \
0  혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임,...   
1               혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)   
2           Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)   
3     밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)   
4                                                파이썬   

                                                link  \
0  https://search.shopping.naver.com/book/catalog...   
1  https://search.shopping.naver.com/book/catalog...   
2  https://search.shopping.naver.com/book/catalog...   
3  https://search.shopping.naver.com/book/catalog...   
4  https://search.shopping.naver.com/book/catalog...   

                                               image  author  discount  \
0  https://shopping-phinf.pstatic.net/main_528593...     문현일     23400   
1  https://shopping-phinf.pstatic.net/main_325076...     윤인성     19800   
2  https://shopping-phinf.pstatic.net/main_403540...     박응용     19800   
3  https://shopping-phinf.pstatic.net/main_526155...  사이토 고키     23400   
4  https://shopping-phinf.pstatic.net/main_324609...     홍의경     29440   

  publisher   pubdate           isbn  \
0     한빛미디어  20250214  9791169213271   
1     한빛미디어  20220601  9791162245651   
2   이지스퍼블리싱  20230615  9791163034735   
3     한빛미디어  20250124  9791169213387   
4      생능출판  20220309  9788970506784   

                                         description  
0  “파이썬을 배워서 어디에 활용할 수 있을지 궁금했나요?”\n나의 신용카드 내역 분석...  
1  혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서\n\n『혼자 ...  
2  프로그래밍 분야 8년 연속 베스트셀러!\n《Do it! 점프 투 파이썬》 전면 개정...  
3  딥러닝 분야 부동의 베스트셀러!\n머리로 이해하고 손으로 익히는 가장 쉬운 딥러닝 ...  
4  코딩 실력을 키우는 지름길은 실습이다.\n\n2008년 즈음에 검인정 중학교 교과서...

In [4]:

books_df.loc[books_df['discount'] >= 20000,['title','author','discount','publisher','pubdate']]\
    .sort_values(by='discount', ascending=False).reset_index(drop=True)

title           author  \
0                   으뜸 파이썬 (프로그래밍을 사랑하는 두 교수가 작정하고 쓴)          박동규^강영민   
1                                                 파이썬  Y. Daniel Liang   
2                                        으뜸 파이썬 (개정판)          박동규^강영민   
3                            파이썬 마스터 (실생활 융합 예제로 배우는)          김종훈^김동건   
4   파이썬 머신러닝 완벽 가이드 (다양한 캐글 예제와 함께 기초 알고리즘부터 최신 기법...              권철민   
5                                                 파이썬              홍의경   
6                                              독학 파이썬         야마다 요시히로   
7                                             새내기 파이썬              천인국   
8                                             파이썬의 정석          조용주^임좌상   
9                    디딤돌 파이썬 (구조적 프로그래밍으로 설명한 파이썬 기초)              이찬수   
10                     파이썬 + AI (코드조각과 실행화면으로 배우는 코딩)          안용제^김남규   
11                          파이썬의 신 (프로그래밍 언어를 배운다는 것)              김주현   
12                    내공의 파이썬 (한 권으로 끝내는 초보자용 코딩 입문서)              한선관   
13                                            파이썬 플러스              최희식   
14                                                파이썬              김영천   
15     밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)           사이토 고키   
16                                       파이썬 연습 (개정판)              김기용   
17  혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임,...              문현일   
18                             파이썬 프로그래밍 (기초부터 딥러닝까지)               허준   
19                              엑셀X파이썬 (엑셀이 파이썬을 품었다)              정성일   
20                          파이썬 입문 (3step으로 확실하게 배우는)  야마다 요시히로^야마다 나미   
21                         파이썬 (다양한 예제로 알로리즘 쉽게 분석하기)              김유진   
22   코딩 자율학습 나도코딩의 파이썬 입문 (초보자 눈높이에 맞춘 친절한 프로그래밍 자습서)             나도코딩   
23                                      파이썬을 이용한 통계수학              권세혁   
24                             파이썬 올인원 (기초부터 데이터분석까지)          이지민^임희경   

    discount publisher   pubdate  
0      31680      생능출판  20200217  
1      31500     에피스테메  20180302  
2      31280      생능출판  20240614  
3      31040    한빛아카데미  20250117  
4      29800      위키북스  20220421  
5      29440      생능출판  20220309  
6      28800     정보문화사  20241220  
7      27600      생능출판  20220630  
8      26820     길벗캠퍼스  20230110  
9      26100    인피니티북스  20220823  
10     25200     디지털북스  20240715  
11     24300       로드북  20161223  
12     24300    자유아카데미  20250315  
13     23750    인피니티북스  20240731  
14     23750       기한재  20200225  
15     23400     한빛미디어  20250124  
16     23400        그린  20220225  
17     23400     한빛미디어  20250214  
18     22500       성안당  20250312  
19     22500        길벗  20240730  
20     22500    시그마프레스  20210105  
21     22500        다본  20190410  
22     21600        길벗  20230220  
23     21560    자유아카데미  20250315  
24     20700     연두에디션  20230904

In [6]:
# 함수로 정의
def filter_and_sort_books(df, min_discount=20000):
    """
    할인 금액이 min_discount 이상인 도서 필터링 후 정렬
    
    Parameters:
        df (DataFrame): 도서 데이터프레임
        min_discount (int): 최소 할인 금액 기준 (기본값 20000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['discount'] >= min_discount, ['title', 'author', 'discount', 'publisher', 'pubdate']]
          .sort_values(by='discount', ascending=False)
          .reset_index(drop=True)
    )

filter_and_sort_books(books_df,30000)

title           author  discount publisher  \
0  으뜸 파이썬 (프로그래밍을 사랑하는 두 교수가 작정하고 쓴)          박동규^강영민     31680      생능출판   
1                                파이썬  Y. Daniel Liang     31500     에피스테메   
2                       으뜸 파이썬 (개정판)          박동규^강영민     31280      생능출판   
3           파이썬 마스터 (실생활 융합 예제로 배우는)          김종훈^김동건     31040    한빛아카데미   

    pubdate  
0  20200217  
1  20180302  
2  20240614  
3  20250117

In [7]:
# image , description 컬럼은 제외한 모든 컬럼 출력하기
books_df.loc[books_df['publisher'].str.contains('인피니티북스'),books_df.columns.drop(['image','description'])].reset_index(drop=True)

title  \
0                               파이썬   
1                           파이썬 플러스   
2  디딤돌 파이썬 (구조적 프로그래밍으로 설명한 파이썬 기초)   

                                                link author  discount  \
0  https://search.shopping.naver.com/book/catalog...    천인국     12000   
1  https://search.shopping.naver.com/book/catalog...    최희식     23750   
2  https://search.shopping.naver.com/book/catalog...    이찬수     26100   

  publisher   pubdate           isbn  
0    인피니티북스  20170830  9791185578330  
1    인피니티북스  20240731  9791192373362  
2    인피니티북스  20220823  9791192373058

In [8]:
books_df['publisher'].unique()

array(['한빛미디어', '이지스퍼블리싱', '생능출판', '렉스미디어닷넷', '북두', '인피니티북스', '에듀웨이',
       '복두출판사', '기한재', '다본', '에피스테메', '위키북스', '자유아카데미', '정보문화사', '한빛아카데미',
       '길벗', '디지털북스', '성안당', '클라우드북스', '퍼플', '그린', '북랩', '에이콘출판',
       '시그마프레스', '지오북스', '이모션미디어', '로드북', '연두에디션', '시대인', '길벗캠퍼스'],
      dtype=object)

In [9]:
def filter_books_by_publisher(df, publisher_name):
    """
    특정 출판사가 포함된 도서만 필터링 (image, description 컬럼 제외)

    Parameters:
        df (DataFrame): 도서 데이터프레임
        publisher_name (str): 포함할 출판사 이름

    Returns:
        DataFrame: 필터링된 결과
    """
    return (
        df.loc[df['publisher'].str.contains(publisher_name), df.columns.drop(['image', 'description'])]
          .reset_index(drop=True)
    )

filter_books_by_publisher(books_df,'한빛미디어')

title  \
0  혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임,...   
1               혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)   
2     밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)   
3                     파이썬 3 (프로그래밍을 배우기에 가장 재미있는 언어)   
4          파이썬 정복 (파이썬 개발에 필요한 기본 지식을 모두 담은 파이썬 입문서)   

                                                link  author  discount  \
0  https://search.shopping.naver.com/book/catalog...     문현일     23400   
1  https://search.shopping.naver.com/book/catalog...     윤인성     19800   
2  https://search.shopping.naver.com/book/catalog...  사이토 고키     23400   
3  https://search.shopping.naver.com/book/catalog...     박상현         0   
4  https://search.shopping.naver.com/book/catalog...     김상형     19800   

  publisher   pubdate           isbn  
0     한빛미디어  20250214  9791169213271  
1     한빛미디어  20220601  9791162245651  
2     한빛미디어  20250124  9791169213387  
3     한빛미디어  20160210  9788968482359  
4     한빛미디어  20180402  9791162240540

In [10]:
import pandas as pd

shops_df = pd.read_json('../data/shops.json')
print(type(shops_df))
shops_df.head(2)

<class 'pandas.core.frame.DataFrame'>


title  \
0  백화점3-5일배송 폴로 <b>가디건</b> 걸즈 걸스 랄프로렌 꽈배기 블랙 화이트 ...   
1  봄 가을 간절기 라운드 루즈핏 크롭 <b>가디건</b> 레드 그린 핑크 화이트 블랙...   

                                                link  \
0  https://smartstore.naver.com/main/products/667...   
1  https://smartstore.naver.com/main/products/546...   

                                               image  lprice hprice mallName  \
0  https://shopping-phinf.pstatic.net/main_842166...   64900           나우인뉴욕   
1  https://shopping-phinf.pstatic.net/main_830114...   19900           ARUMY   

     productId  productType   brand   maker category1 category2 category3  \
0  84216650390            2  폴로랄프로렌  폴로랄프로렌      패션의류      여성의류       카디건   
1  83011442056            2                      패션의류      여성의류       카디건   

  category4  
0            
1

In [11]:
shops_df.loc[shops_df['lprice'] <= 50000,['brand','lprice','mallName','link']]\
    .sort_values(by='lprice').reset_index(drop=True)

brand  lprice   mallName  \
0              13100        네이버   
1        H&M   17512        네이버   
2              18900       코이블리   
3              18900       코이블리   
4              19800      로이앤한나   
5              19900      ARUMY   
6        리스트   23000        네이버   
7              23900       고고하니   
8              24800     베이지블랑.   
9              24900      ARUMY   
10       리스트   24940        네이버   
11             27000    건강한생활82   
12  비비안웨스트우드   28070        네이버   
13             29800  미드시티 여성니트   
14             29900       지읒이응   
15   제너럴아이디어   31416        네이버   
16   제너럴아이디어   32400        네이버   
17      쉬즈미스   36910        네이버   
18       벤시몽   37650        네이버   
19        사색   39700         사색   
20             39800       왓위웨어   
21             42000    ttoyuni   
22        사색   43700         사색   
23       벤시몽   44910        네이버   
24        로엠   44910        네이버   
25             49500       루나드엘   

                                                 link  
0   https://search.shopping.naver.com/catalog/5369...  
1   https://search.shopping.naver.com/catalog/5178...  
2   https://smartstore.naver.com/main/products/482...  
3   https://smartstore.naver.com/main/products/438...  
4   https://smartstore.naver.com/main/products/490...  
5   https://smartstore.naver.com/main/products/546...  
6   https://search.shopping.naver.com/catalog/5114...  
7   https://smartstore.naver.com/main/products/559...  
8   https://smartstore.naver.com/main/products/519...  
9   https://smartstore.naver.com/main/products/995...  
10  https://search.shopping.naver.com/catalog/4958...  
11  https://smartstore.naver.com/main/products/798...  
12  https://search.shopping.naver.com/catalog/4347...  
13  https://smartstore.naver.com/main/products/100...  
14  https://smartstore.naver.com/main/products/539...  
15  https://search.shopping.naver.com/catalog/4954...  
16  https://search.shopping.naver.com/catalog/5328...  
17  https://search.shopping.naver.com/catalog/4962...  
18  https://search.shopping.naver.com/catalog/5337...  
19  https://smartstore.naver.com/main/products/115...  
20  https://smartstore.naver.com/main/products/529...  
21  https://smartstore.naver.com/main/products/904...  
22  https://smartstore.naver.com/main/products/113...  
23  https://search.shopping.naver.com/catalog/5323...  
24  https://search.shopping.naver.com/catalog/5267...  
25  https://smartstore.naver.com/main/products/997...

In [12]:
def filter_and_sort_shops(df, max_price=50000):
    """
    최대 가격 이하 상품 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        max_price (int): 최대 가격 기준 (기본값 50000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['lprice'] <= max_price, ['brand', 'lprice', 'mallName', 'link']]
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_and_sort_shops(shops_df,40000)

brand  lprice   mallName  \
0              13100        네이버   
1        H&M   17512        네이버   
2              18900       코이블리   
3              18900       코이블리   
4              19800      로이앤한나   
5              19900      ARUMY   
6        리스트   23000        네이버   
7              23900       고고하니   
8              24800     베이지블랑.   
9              24900      ARUMY   
10       리스트   24940        네이버   
11             27000    건강한생활82   
12  비비안웨스트우드   28070        네이버   
13             29800  미드시티 여성니트   
14             29900       지읒이응   
15   제너럴아이디어   31416        네이버   
16   제너럴아이디어   32400        네이버   
17      쉬즈미스   36910        네이버   
18       벤시몽   37650        네이버   
19        사색   39700         사색   
20             39800       왓위웨어   

                                                 link  
0   https://search.shopping.naver.com/catalog/5369...  
1   https://search.shopping.naver.com/catalog/5178...  
2   https://smartstore.naver.com/main/products/482...  
3   https://smartstore.naver.com/main/products/438...  
4   https://smartstore.naver.com/main/products/490...  
5   https://smartstore.naver.com/main/products/546...  
6   https://search.shopping.naver.com/catalog/5114...  
7   https://smartstore.naver.com/main/products/559...  
8   https://smartstore.naver.com/main/products/519...  
9   https://smartstore.naver.com/main/products/995...  
10  https://search.shopping.naver.com/catalog/4958...  
11  https://smartstore.naver.com/main/products/798...  
12  https://search.shopping.naver.com/catalog/4347...  
13  https://smartstore.naver.com/main/products/100...  
14  https://smartstore.naver.com/main/products/539...  
15  https://search.shopping.naver.com/catalog/4954...  
16  https://search.shopping.naver.com/catalog/5328...  
17  https://search.shopping.naver.com/catalog/4962...  
18  https://search.shopping.naver.com/catalog/5337...  
19  https://smartstore.naver.com/main/products/115...  
20  https://smartstore.naver.com/main/products/529...

In [13]:
shops_df.loc[shops_df['mallName'] == '네이버','lprice':'brand']\
    .sort_values(by='lprice').reset_index(drop=True)

lprice hprice mallName    productId  productType     brand
0    13100             네이버  53690259105            1          
1    17512             네이버  51781610172            1       H&M
2    23000             네이버  51143733510            1       리스트
3    24940             네이버  49580876217            1       리스트
4    28070             네이버  43475137325            1  비비안웨스트우드
5    31416             네이버  49541497760            1   제너럴아이디어
6    32400             네이버  53287208769            1   제너럴아이디어
7    36910             네이버  49625464649            1      쉬즈미스
8    37650             네이버  53373694627            1       벤시몽
9    44910             네이버  53232763110            1       벤시몽
10   44910             네이버  52675721484            1        로엠
11   80100             네이버  49714141886            1      라코스테
12   85980             네이버  53126962083            1          
13  143100             네이버  49912758699            1      라코스테
14  174090             네이버  50705955349            1      라코스테

In [14]:
shops_df['mallName'].unique()

array(['나우인뉴욕', 'ARUMY', '트위티 155', '코이블리', '세컨찬스라이프', '레이바쿠', '브랜드리퍼블릭',
       '바이찬스', '뉴욕바잉', '루나드엘', '네이버', '리틀데이즈', '브랜드사는이쁜언니', '미드시티 여성니트',
       'ttoyuni', '지읒이응', '트렌드홀릭', '버닝웨어', '고고하니', '건강한생활82', '베이지블랑.',
       '사색', '부스더샵', '왓위웨어', '포니홀릭', '도쿄루꼬', '로이앤한나', '빨간단풍 구매대행'],
      dtype=object)

In [15]:
shops_df['brand'].unique()

array(['폴로랄프로렌', '', '꼼데가르송', '메종키츠네', '폴로', '제너럴아이디어', 'H&M', '라코스테',
       '톰브라운', '사색', '벤시몽', '리스트', '비비안웨스트우드', '쉬즈미스', '로엠', '타미힐피거'],
      dtype=object)

In [16]:
def filter_shops_by_mall(df, mall_name='네이버'):
    """
    특정 쇼핑몰 상품만 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        mall_name (str): 쇼핑몰 이름 (기본값 '네이버')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['mallName'] == mall_name, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops_by_mall(shops_df,'나우인뉴욕')

lprice hprice mallName    productId  productType   brand
0   64900           나우인뉴욕  84216650390            2  폴로랄프로렌

#### mallName 또는 brand 로 검색하기

In [17]:
def filter_shops(df, keyword, search_type='mall'):
    """
    mallName 또는 brand 기준으로 필터링 후 가격 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        keyword (str): 검색할 값 (예: '네이버', '폴로랄프로렌')
        search_type (str): 'mall' or 'brand' (기본값 'mall')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    
    if search_type == 'mall':
        condition = df['mallName'] == keyword
    elif search_type == 'brand':
        condition = df['brand'] == keyword
    else:
        raise ValueError("search_type은 'mall' 또는 'brand'만 가능합니다.")

    return (
        df.loc[condition, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops(shops_df,'라코스테','brand')

lprice hprice mallName    productId  productType brand
0   80100             네이버  49714141886            1  라코스테
1  143100             네이버  49912758699            1  라코스테
2  174090             네이버  50705955349            1  라코스테